In [1]:
import numpy as np
import scipy as sp
import torch
import matplotlib.pyplot as plt

plt.rcParams['axes.facecolor'] = 'white'


import sys
sys.path.append('conditional_gan/')

import datetime
import warnings


In [2]:
warnings.filterwarnings('ignore')

In [3]:
import torch.utils.data
import os.path
import random
import torchvision.transforms as transforms
import numpy as np
from numpy import inf
import matplotlib.gridspec as gridspec
import networkx
from skimage.transform import rescale, resize, downscale_local_mean
from matplotlib.pyplot import figure
import seaborn as sns


In [4]:
import netCDF4 as nc4

In [10]:
lnd_data = 'Livneh.LndVar.CA.4km.2003_2013.nc'
atm_data= 'Livneh.AtmVar.CA.4km.2003_2013.nc'

In [11]:
f_atm = nc4.Dataset(atm_data,'r')
f_lnd = nc4.Dataset(lnd_data,'r')

IOError: [Errno 2] No such file or directory: 'Livneh.AtmVar.CA.4km.2003_2013.nc'